In [84]:
# criar um navegador
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# importar/visualizar a base de dados
pd.read_excel('buscas.xlsx')    

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


Definição das funções de busca no google e no buscape

In [110]:

def find_google(driver, product, banned_terms, min_price, max_price):
    list_google = []

    banned_terms = banned_terms.lower()
    list_banned_terms = banned_terms.split(' ')
    list_product_terms = product.split(' ')


    # pesquisar o nome do produto no google
    driver.get('https://www.google.com.br')
    sleep(1)
    # clicar na aba shopping
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[1]/div[2]/textarea').send_keys(product, Keys.ENTER)

    sleep(1)
    list_items = driver.find_elements(By.XPATH, '//div[@role="listitem"]')

    sleep(1)
    for item in list_items:
        if 'Shopping' in item.text:
            item.click()
            break
    sleep(1)
    # pegar a lista de resultados da busca no google shopping

    list_results = WebDriverWait(driver, 1).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'i0X6df'))
    )

    # Processa os resultados
    for result in list_results:
        try:
            # Pega o nome do produto
            name = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.TAG_NAME, 'h3'))).text.lower()

            # Verifica se o nome contém termos banidos
            banned_terms_found = any(word in name for word in list_banned_terms)
            
            # Verifica se o nome contém todos os termos do produto
            product_terms_found = all(word in name for word in list_product_terms)

            # Se o nome não contém termos banidos e contém os termos do produto
            if not banned_terms_found and product_terms_found:
                # Pega o preço do produto
                price = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'a8Pemb'))).text
                price = float(price.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))

                # Verifica se o preço está dentro da faixa desejada
                if min_price <= price <= max_price:
                    # Pega o link do produto
                    link = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'Lq5OHe'))).get_attribute('href')
                    list_google.append((name, price, link))
        except Exception as e:
            print(f"Erro ao processar um resultado: {e}")
    driver.quit()    
    return list_google


In [114]:
# criar o navegador
driver = webdriver.Chrome()
# tratar os valores que vieram da tabela
product = 'iphone 12 64 gb'
banned_terms = 'mini watch'
min_price = 3000
max_price = 3500

list_google = find_google(driver, product, banned_terms, min_price, max_price)
print(list_google)

[]


In [115]:
len(list_google)

0

Contrução da nossa lista de ofertas encontradas

Exportar a base de ofertas para Excel

In [82]:
# exportar pro excel



Enviando o e-mail

In [83]:
# enviar por e-mail o resultado da tabela


#verificando se existe alguma oferta dentro da tabela de ofertas
